In [1]:

from analysis.probability import simple_posneg as probability_generator
from evaluate import evaluate_single

def eval_simple(note):
    return evaluate_single(probability_generator(note), root_note=note)

In [2]:
import numpy as np
from analysis.musical import consonance_ordered_notes, empirical_probabilities
from analysis.spike_tensor import generate_spike_tensor
from cache import get_spikes
from evaluate import predicted_consonance_scores, predicted_probabilities

root_note = "C4"
notes = consonance_ordered_notes(root_note)
notes_spikes = [get_spikes(note) for note in notes]
consonance_ordered_tensors = [generate_spike_tensor(spikes) for spikes in notes_spikes]

root_tensor = generate_spike_tensor(get_spikes(root_note))
probability_tensor = probability_generator(root_note)

def try_eval(tensors, metric, debug=False):
    scores = predicted_consonance_scores(probability_tensor, tensors, root_tensor)

    P_predicted = predicted_probabilities(scores)

    if debug:
        def print_arr(arr):
            print(np.round(np.array(arr), 3))
        print("Scores")
        print_arr(scores)
        print("P_predicted")
        print_arr(P_predicted)
        print("P_empirical")
        print_arr(empirical_probabilities())

    print(metric(P_predicted))

In [3]:
from evaluate import js_divergence

try_eval(consonance_ordered_tensors, js_divergence, debug=True)

Scores
[-0.615 -0.687 -0.689 -0.695 -0.696 -0.666 -0.688 -0.67  -0.672 -0.649
 -0.689 -0.678]
P_predicted
[0.312 0.036 0.03  0.004 0.    0.114 0.032 0.1   0.095 0.18  0.028 0.069]
P_empirical
[0.147 0.133 0.12  0.107 0.107 0.093 0.08  0.067 0.053 0.053 0.027 0.013]
0.3869871813546658


In [4]:
import numpy as np

def shuffle_attempts(metric, N=10):
    data = []
    for i in range(N):
        A = consonance_ordered_tensors[0][0][0]
        np.random.shuffle(consonance_ordered_tensors)
        B = consonance_ordered_tensors[0][0][0]
        if not np.array_equal(A, B):
            value = try_eval(consonance_ordered_tensors, metric)
            data.append(value)

In [5]:
from evaluate import js_divergence

shuffle_attempts(js_divergence)

0.38660410674711476
0.3626837146922029
0.3336491446707358
0.427786582941451
0.4461984739198874
0.3782307454264301
0.40558715988552435
0.45475612420380324
0.36494136741310074


In [10]:
import os

from cache import SPIKES_DIR


root_note = "C4"
notes = consonance_ordered_notes(root_note)
notes = [notes[0], notes[-1]]
notes_spikes = [get_spikes(note) for note in notes]
tensors = [generate_spike_tensor(spikes) for spikes in notes_spikes]

white_noise_spikes = np.load(os.path.join(SPIKES_DIR, "white_noise.npy"))
tensors.append(generate_spike_tensor(white_noise_spikes))

root_tensor = generate_spike_tensor(get_spikes(root_note))

scores = predicted_consonance_scores(probability_tensor, tensors, root_tensor)
print("Scores")
print(np.round(np.array(scores), 3))

P_predicted = predicted_probabilities(scores)
print("P_predicted")
print(np.round(np.array(P_predicted), 2))

print("Most Cons, Least Cons, White Noise")


Scores
[-0.615 -0.678 -1.268]
P_predicted
[0.53 0.47 0.  ]
Most Cons, Least Cons, White Noise


In [7]:
keys = ["C4", "C#4", "D4", "D#4", "E4", "F4", "F#4", "G4", "G#4", "A4", "A#4", "B4"]

for key in keys:
    print(f"{key}: {eval_simple(key)}")

C4: 61
C#4: 55
D4: 59
D#4: 59
E4: 68
F4: 61
F#4: 64
G4: 70
G#4: 69
A4: 70
A#4: 62
B4: 75


In [8]:
consonance_ordered_notes(root_note)

['C5', 'G4', 'F4', 'E4', 'D#4', 'A4', 'G#4', 'D4', 'C#4', 'B4', 'A#4', 'F#4']

In [9]:
# collect_data(kl_divergence_reverse)
# collect_data(kl_divergence)